In [7]:
## Traditional Oven Function

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from BodyModelling import *

rockwool = RockWool()
steel = Steel()
air = Air()
chicken = Chicken()
door = Door()

def sensitivity(params):

    bodies = {}
    bodies['fanair'] = Gas(290, 0.002, air, 25)
    bodies['ovenair'] = Gas(290, 0.05, air, 15)
    bodies['fanshroud'] = Solid(290, 0.00006, steel, 0.001)
    bodies['ovenback'] = Solid(290, 0.00012, steel, 0.001)
    bodies['ovenbackinsulation'] = Solid(290, 0.0024, rockwool, 0.02)
    bodies['ovenbackouter'] = Solid(290, 0.00012, steel, 0.001)

    #bodies['food'] = Solid(277, 0.0036, chicken, 0.04) # chicken
    bodies['food'] = Solid(277, 0.00071, chicken, 0.005) # pizza


    bodies['ovenwalls'] = Solid(290, 0.00072, steel, 0.001)
    bodies['oveninsulation'] = Solid(290, 0.0144, rockwool, 0.02)
    bodies['ovenouter'] = Solid(290, 0.00072, steel, 0.001)
    bodies['ovendoor'] = Solid(290, 0.006, door, 0.03)
    bodies['outsideair'] = Surroundings(290, 50)
    bodies['outsideairback'] = Surroundings(290, 50)


    exchangers = []
    exchangers.append(MassExchange(bodies['fanair'], bodies['ovenair'], 0.002))
    exchangers.append(HeatExchange(bodies['fanair'], bodies['ovenback'], 0.06))
    exchangers.append(HeatExchange(bodies['fanair'], bodies['fanshroud'], 0.06))
    exchangers.append(HeatExchange(bodies['ovenback'], bodies['ovenbackinsulation'], 0.2))
    exchangers.append(HeatExchange(bodies['ovenbackinsulation'], bodies['ovenbackouter'], 0.2))
    exchangers.append(HeatExchange(bodies['ovenback'], bodies['ovenair'], 0.06))
    exchangers.append(HeatExchange(bodies['fanshroud'], bodies['ovenair'], 0.06))
    exchangers.append(HeatExchange(bodies['ovenair'], bodies['food'], 0.13))
    exchangers.append(HeatExchange(bodies['ovenair'], bodies['ovenwalls'], 0.72))
    exchangers.append(HeatExchange(bodies['ovenair'], bodies['ovendoor'], 0.2))
    exchangers.append(HeatExchange(bodies['ovenair'], bodies['ovenback'], 0.2-0.06))
    exchangers.append(HeatExchange(bodies['ovenwalls'], bodies['oveninsulation'], 0.72))
    exchangers.append(HeatExchange(bodies['oveninsulation'], bodies['ovenouter'], 0.72))
    exchangers.append(HeatExchange(bodies['ovenouter'], bodies['outsideair'], 0.72))
    exchangers.append(HeatExchange(bodies['ovenbackouter'], bodies['outsideairback'], 0.72))
    exchangers.append(HeatExchange(bodies['ovendoor'], bodies['outsideair'], 0.2))


    step_time = params[0]
    steps = 1E9 # Contingency
    step_count = 0
    setpoint = 180+273
    foodtarget = 100+273
    power = 2400 #Watts
    savename = "Traditional Oven Chicken"
    results = {}
    results["time"] = []
    results["energy"] = []
    for body in bodies:
        results[body] = []


    while step_count<steps and bodies["food"].temp<foodtarget:
        #print "step: ", step_count
        if(bodies['ovenair'].temp<setpoint):
            bodies["fanair"].add_flux(2400)
            results["energy"].append(2400*step_time)
        else:
            results["energy"].append(0)
        for exchanger in exchangers:
            exchanger.calc_flux()
        for body in bodies:
            results[body].append(bodies[body].temp-273)
            bodies[body].time_step(step_time)        
            #print body, bodies[body].temp
        results["time"].append(step_time*step_count)
        step_count+=1

    return results["time"][-1]/60, sum(results["energy"])/1000/3600

In [8]:
timesteps = [0.001,0.005,0.01,0.05,0.1,0.2,0.5,1]
results = {}
results["time"] = []
results["energy"] = []
results["time step"] = []
for timestep in timesteps:
    res = sensitivity([timestep])
    results["time step"].append(timestep)
    results["time"].append(res[0])
    results["energy"].append(res[1])
timestep_results = pd.DataFrame(results)
timestep_results.to_csv(r"Results/TradPizzaTimestepSensitivityResults.csv")


In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from BodyModelling import *

def sensitivity(params):
    rockwool = RockWool()
    steel = Steel()
    air = Air()
    chicken = Chicken()
    door = Door()

    bodies = {}
    bodies['jetsair'] = Gas(290, 0.002, air, 35)
    bodies['impingementair'] = Gas(290, 0.002, air, 35)
    bodies['ovenair'] = Gas(290, 0.05, air, 15)
    bodies["ductair"] = Gas(290, 0.002, air, 15)
    bodies["jetsductwork"] = Solid(290, 0.00006, steel, 0.001)
    bodies['backductwork'] = Solid(290, 0.00006, steel, 0.001)
    bodies['ovenback'] = Solid(290, 0.00012, steel, 0.001)
    bodies['ovenbackinsulation'] = Solid(290, 0.0024, rockwool, 0.02)
    bodies['ovenbackouter'] = Solid(290, 0.00012, steel, 0.001)

    #bodies['food'] = Solid(277, 0.0036, chicken, 0.04) # chicken
    bodies['food'] = Solid(277, 0.00071, chicken, 0.005) # pizza


    bodies['ovenwalls'] = Solid(290, 0.00072, steel, 0.001)
    bodies['oveninsulation'] = Solid(290, 0.0144, rockwool, 0.02)
    bodies['ovenouter'] = Solid(290, 0.00072, steel, 0.001)
    bodies['ovendoor'] = Solid(290, 0.006, door, 0.03)
    bodies['outsideair'] = Surroundings(290, 50)
    bodies['outsideairback'] = Surroundings(290, 50)

    mixing_ratio = 0.5# amount of jetsair in impingmentair

    exchangers = []
    exchangers.append(MassExchange(bodies['jetsair'], bodies['ovenair'], 0.002))
    exchangers.append(MassExchange(bodies['ovenair'], bodies['ductair'], 0.002))
    exchangers.append(MassExchange(bodies['ductair'], bodies['jetsair'], 0.002))

    exchangers.append(MassExchange(bodies['jetsair'], bodies['impingementair'], 0.002*mixing_ratio))
    exchangers.append(MassExchange(bodies['ovenair'], bodies['impingementair'], 0.002*(1-mixing_ratio)))
    exchangers.append(MassExchange(bodies['impingementair'], bodies['ovenair'], 0.002))

    exchangers.append(HeatExchange(bodies['ductair'], bodies['ovenback'], 0.06))
    exchangers.append(HeatExchange(bodies['ductair'], bodies['backductwork'], 0.06))

    exchangers.append(HeatExchange(bodies['ovenair'], bodies['backductwork'], 0.06))

    exchangers.append(HeatExchange(bodies['jetsair'], bodies['jetsductwork'], 0.06))
    exchangers.append(HeatExchange(bodies['ovenair'], bodies['jetsductwork'], 0.06))

    exchangers.append(HeatExchange(bodies['ovenback'], bodies['ovenbackinsulation'], 0.12))
    exchangers.append(HeatExchange(bodies['ovenbackinsulation'], bodies['ovenbackouter'], 0.12))
    exchangers.append(HeatExchange(bodies['ovenback'], bodies['ovenair'], 0.06))
    exchangers.append(HeatExchange(bodies['impingementair'], bodies['food'], 0.13))
    exchangers.append(HeatExchange(bodies['ovenair'], bodies['ovenwalls'], 0.344))
    exchangers.append(HeatExchange(bodies['ovenwalls'], bodies['oveninsulation'], 0.344))
    exchangers.append(HeatExchange(bodies['oveninsulation'], bodies['ovenouter'], 0.344))
    exchangers.append(HeatExchange(bodies['ovenouter'], bodies['outsideair'], 0.344))
    exchangers.append(HeatExchange(bodies['ovenback'], bodies['outsideairback'], 0.12))
    exchangers.append(HeatExchange(bodies['ovendoor'], bodies['outsideair'], 0.2))
    exchangers.append(HeatExchange(bodies['ovenair'], bodies['ovendoor'], 0.2))

    step_time = 0.01
    steps = 1E10
    step_count = 0
    setpoint = 180+273
    foodtarget = 100+273
    power = 2400 #Watts
    savename = "COI Oven Chicken Mixing0"
    results = {}
    results["time"] = []
    results["energy"] = []
    for body in bodies:
        results[body] = []


    while step_count<steps and bodies["food"].temp<foodtarget:
        #print "step: ", step_count
        if(bodies['ovenair'].temp<setpoint):
            bodies["jetsair"].add_flux(2400)
            results["energy"].append(2400*step_time)
        else:
            results["energy"].append(0)
        for exchanger in exchangers:
            exchanger.calc_flux()
        for body in bodies:
            results[body].append(bodies[body].temp-273)
            bodies[body].time_step(step_time)        
            #print body, bodies[body].temp
        results["time"].append(step_time*step_count)
        step_count+=1
    return results["time"][-1]/60, sum(results["energy"])/1000/3600

In [11]:
timesteps = [0.001,0.005,0.01,0.05,0.1,0.2,0.5,1]
results = {}
results["time"] = []
results["energy"] = []
results["time step"] = []
for timestep in timesteps:
    res = sensitivity([timestep])
    results["time step"].append(timestep)
    results["time"].append(res[0])
    results["energy"].append(res[1])
timestep_results = pd.DataFrame(results)
timestep_results.to_csv(r"Results/COIPizzaTimestepSensitivityResults.csv")